# <h1 style='font-family:Lato,sans-serif;color:#ca5800;text-align:center;'>Machine Learning Model Training

<span style='font-family:Lato,sans-serif;'>author: <span style='color:#1696d2'>[@baiochi](#https://github.com/baiochi/)

<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.2em;'> Load libraries and functions

In [2]:
# Libraries, functions and custom configurations
from config.custom import *
# Load CSS for custom Markdown
HTML(open('./config/custom.css', 'r').read())

Numpy: 1.20.3
Pandas: 1.3.4
Sklearn: 1.0.2
Matplotlib: 3.4.3
Seaborn: 0.11.2


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'>Analysis Setup

In [5]:
configuration = {
	# Setup file locations
	'df_path' : 		'./data/',
	'df_name' : 		'application_train.csv',
	'pickle_path' : 	'pickle_data/first_cycle/',
	# Select estimator
	'estimator' : 		XGBClassifier,
	# Select parameters
	'estimator_params' : {
						'learning_rate' : np.linspace(0.00001,0.1,100),
						'n_estimators' : range(500, 1000),
						'min_child_weight': [1, 5, 10],
						'gamma': [0.5, 1, 1.5, 2, 5],
						'subsample': [0.6, 0.8, 1.0],
						'colsample_bytree': [0.6, 0.8, 1.0],
						'max_depth': [3, 4, 5,6]
						},
	# Select size of dataset to split
	'data_size' : 		100,
	# Select metric of evaluation
	'metric' : 			'roc_auc',
	# Select type of Scoring ['classification', 'regression', 'clustering']
	'metric_type' : 	'classification',
	# Select number of hyperopt iterations
	'max_evals' : 		10
}


"""
RandomForestClassifier
{
						'criterion' : ['gini', 'entropy'],
						'n_estimators' : range(1, 501),
						'max_depth' : [1, 2]
						},
"""

"\nRandomForestClassifier\n{\n\t\t\t\t\t\t'criterion' : ['gini', 'entropy'],\n\t\t\t\t\t\t'n_estimators' : range(1, 501),\n\t\t\t\t\t\t'max_depth' : [1, 2]\n\t\t\t\t\t\t},\n"

In [6]:
# Assign new variables for a cleaner code
df_path = configuration['df_path']
df_name = configuration['df_name']
pickle_path = configuration['pickle_path']
estimator = configuration['estimator']
estimator_params = configuration['estimator_params']
data_size = configuration['data_size']
train_size, test_size = select_df_split_size(data_size)
metric = configuration['metric']
metric_type = configuration['metric_type']
max_evals = configuration['max_evals']

<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Load Data

In [7]:
df = pd.read_csv(df_path + df_name)
print(f'{CYAN}Dataset info{WHITE}\nObservations: {df.shape[0]}\nFeatures: {df.shape[1]}')
df.head()

Dataset info
Observations: 246008
Features: 122


SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      456162       0         Cash loans           F            N   
1      134978       0         Cash loans           F            N   
2      318952       0         Cash loans           M            Y   
3      361264       0         Cash loans           F            N   
4      260639       0         Cash loans           F            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               N             0        112500.000  700830.000    22738.500   
1               N             0         90000.000  375322.500    14422.500   
2               N             0        180000.000  544491.000    16047.000   
3               Y             0        270000.000  814041.000    28971.000   
4               Y             0        144000.000  675000.000    21906.000   

   AMT_GOODS_PRICE NAME_TYPE_SUITE      NAME_INCOME_TYPE  \
0       585000.000   Unaccompanied               Working   
1       324000.000   Unaccompanied  Commercial associate   
2       454500.000   Unaccompanied               Working   
3       679500.000   Unaccompanied             Pensioner   
4       675000.000   Unaccompanied               Working   

             NAME_EDUCATION_TYPE    NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  \
0              Incomplete higher  Single / not married  House / apartment   
1  Secondary / secondary special               Married  House / apartment   
2  Secondary / secondary special               Married  House / apartment   
3  Secondary / secondary special               Married  House / apartment   
4  Secondary / secondary special             Separated  House / apartment   

   REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  \
0                       0.020       -8676           -813          -4163.000   
1                       0.025      -13583           -223          -3554.000   
2                       0.036      -13993          -6202          -7971.000   
3                       0.046      -22425         365243         -11805.000   
4                       0.026      -18839          -2763          -5069.000   

   DAYS_ID_PUBLISH  OWN_CAR_AGE  FLAG_MOBIL  FLAG_EMP_PHONE  FLAG_WORK_PHONE  \
0            -1363          NaN           1               1                1   
1            -3287          NaN           1               1                0   
2            -4175        9.000           1               1                1   
3            -1732          NaN           1               0                0   
4            -2381          NaN           1               1                0   

   FLAG_CONT_MOBILE  FLAG_PHONE  FLAG_EMAIL        OCCUPATION_TYPE  \
0                 1           0           0             Core staff   
1                 1           0           0  High skill tech staff   
2                 1           0           0               Managers   
3                 1           1           0                    NaN   
4                 1           1           0               Laborers   

   CNT_FAM_MEMBERS  REGION_RATING_CLIENT  REGION_RATING_CLIENT_W_CITY  \
0            1.000                     2                            2   
1            2.000                     2                            2   
2            2.000                     2                            2   
3            2.000                     1                            1   
4            1.000                     2                            2   

  WEEKDAY_APPR_PROCESS_START  HOUR_APPR_PROCESS_START  \
0                     FRIDAY                       17   
1                     MONDAY                       11   
2                   THURSDAY                       15   
3                    TUESDAY                        9   
4                     FRIDAY                       16   

   REG_REGION_NOT_LIVE_REGION  REG_REGION_NOT_WORK_REGION  \
0                           0                           0   
1             

<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Define target, features and split data

In [8]:
X = df.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y = df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
													train_size=train_size, 
													test_size=test_size, 
													stratify=y, random_state=42)
numeric_features     = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=np.number).columns.tolist()
print(f'{CYAN}Train dataset size:{WHITE} {X_train.shape}')
print(f'{CYAN}Test dataset size:{WHITE} {X_test.shape}')

Train dataset size: (196806, 120)
Test dataset size: (49202, 120)


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Create Baseline Model

In [9]:
numeric_features_pipeline = Pipeline([
    ('impute_num', SimpleImputer(strategy='mean')),
    ('std', MinMaxScaler())
])
categorical_features_pipeline = Pipeline([
    ('impute_cat', SimpleImputer(strategy='constant', fill_value='unknow')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
pre_processing_pipeline = ColumnTransformer([
    ('numeric_transformer', numeric_features_pipeline, numeric_features),
    ('categorical_transformer', categorical_features_pipeline ,categorical_features)
])
pipeline = Pipeline([
    ('pre_processing', pre_processing_pipeline),
    ('rf', estimator(random_state=42))
]).fit(X_train, y_train)

print(f'Score for baseline model with {estimator.__name__}')
display_metrics(X_train, X_test, y_train, y_test, pipeline, metric_type)

[03:30:16] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score for baseline model with XGBClassifier
Train metrics:
roc_auc_score 0.859
f1_score 0.149
Test metrics:
roc_auc_score 0.747
f1_score 0.061


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Hyper-parameter Optimization

In [10]:
# create hyper-parameter space
hps_space = create_hps_space(hps_params=estimator_params)
# create objective function
objective_function = create_fmin_function(estimator=estimator, 
										pp_pipeline=pre_processing_pipeline,
										X_train=X_train, y_train=y_train,
										metric=metric,
										hps_space=hps_space)
# run hyper-parameter search
hps_search = fmin(fn=objective_function, 
					space=hps_space, 
					algo=tpe.suggest, 
					trials=Trials(), 
					rstate=np.random.default_rng(42),
					max_evals=max_evals)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/Users/baiochi/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/baiochi/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/baiochi/opt/anaconda3/lib/python3.9/site-packages/xgboost

[03:36:09] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:36:09] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:36:10] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost

/Users/baiochi/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[04:20:23] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 10%|█         | 1/10 [6:35:46<59:21:55, 23746.17s/trial, best loss: -0.7552526840812375]

...

[16:29:33] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 90%|█████████ | 9/10 [13:18:02<1:00:35, 3635.57s/trial, best loss: -0.7572181789172832]

/Users/baiochi/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/baiochi/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/baiochi/opt/anaconda3/lib/python3.9/site-packages/xgboost

[16:54:04] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:54:12] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:54:12] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost

/Users/baiochi/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:28:03] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
100%|██████████| 10/10 [14:08:53<00:00, 5093.37s/trial, best loss: -0.7572181789172832] 


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Metrics summary

In [11]:
# get results
hps_best_params = space_eval(hps_space, hps_search)
print(f'{CYAN}{estimator.__name__}{WHITE} best parameters:')
for key, value in hps_best_params.items():
        print(f'{GREEN}', key, f'{WHITE}', value)

# create new pipeline with the best parameters
hps_model = Pipeline([('pp', pre_processing_pipeline),
                ('est', estimator(**hps_best_params, random_state=42))]
        ).fit(X_train, y_train)

# print results
print(f'\nScore for best model with hyperopt:')
display_metrics(X_train, X_test, y_train, y_test, hps_model, metric_type)

XGBClassifier best parameters:
 colsample_bytree  0.6
 gamma  0.5
 learning_rate  0.01718
 max_depth  6
 min_child_weight  5
 n_estimators  861
 subsample  0.6
[17:44:48] WARNING: /private/var/folders/_m/3m0bcc4n6h73gny0bt6_g4x80000gn/T/pip-install-4m5tp6_t/xgboost_17c0d7c1e7cb4a3aab80bf74764ec6a2/build/temp.macosx-10.9-x86_64-3.9/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

Score for best model with hyperopt:
Train metrics:
roc_auc_score 0.817
f1_score 0.066
Test metrics:
roc_auc_score 0.760
f1_score 0.039


<h3 style='font-family:Lato,sans-serif;color:#fdbf11;font-size: 1.5em;'> Saving model with Pickle

In [12]:
# calculate how much of data was used in the model training
if data_size < 10:
    data_size = '0' + str(data_size)

# save file
with open(pickle_path + f'{estimator.__name__}_{data_size}_pct.pickle', 'wb') as file:
    pickle.dump(hps_model, file, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'File saved on {os.path.realpath(file.name)}')

File saved on /Users/baiochi/OneDrive/GitHub/LetsCode/5_Machine_Learning/projeto/pickle_data/first_cycle/XGBClassifier_100_pct.pickle
